# Extracting the data out of the zip

In [1]:
import zipfile
import json
import sqlite3
import pandas as pd

conn = sqlite3.connect("scopus.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS papers_raw (
    file_id INTEGER,
    year INTEGER,
    raw_json TEXT
)
""")

conn.commit()
conn.close()

keep = "ScopusData2018-2023/2018/201800000"
zip_path = "ScopusData2018-2023.zip"
years = ["2018", "2019", "2020", "2021", "2022", "2023"]

def loader(year: int, start_id: int, end_id: int):
    conn = sqlite3.connect("scopus.db")
    cur = conn.cursor()

    with zipfile.ZipFile(zip_path, "r") as z:
        for file_id in range(start_id, end_id + 1):
            inner_path = f"ScopusData2018-2023/{year}/{file_id}"
            try:
                with z.open(inner_path) as f:
                    try:
                        obj = json.load(f)
                    except Exception:
                        continue
            except KeyError:
                continue
            raw_text = json.dumps(obj, ensure_ascii=False)
            cur.execute(
                """
                INSERT INTO papers_raw (file_id, year, raw_json)
                VALUES (?, ?, ?)
                """,
                (file_id, year, raw_text)
            )
    conn.commit()
    conn.close()




In [2]:
# 2018
loader(2018, 201800000, 201802761)

# 2019
loader(2019, 201900000, 201903081)

# 2020
loader(2020, 202000000, 202003392)

# 2021
loader(2021, 202100000, 202103814)

# 2022
loader(2022, 202200000, 202204243)

# 2023
loader(2023, 202300000, 202302889)

In [3]:
conn = sqlite3.connect("scopus.db")

df_2018 = pd.read_sql_query(
    """
    SELECT file_id, year, raw_json
    FROM papers_raw
    WHERE year = 2018
    """,
    conn
)

conn.close()

print(df_2018.shape)
print(df_2018.head())

(2762, 3)
     file_id  year                                           raw_json
0  201800000  2018  {"abstracts-retrieval-response": {"item": {"ai...
1  201800001  2018  {"abstracts-retrieval-response": {"item": {"ai...
2  201800002  2018  {"abstracts-retrieval-response": {"item": {"ai...
3  201800003  2018  {"abstracts-retrieval-response": {"item": {"ai...
4  201800004  2018  {"abstracts-retrieval-response": {"item": {"ai...


In [1]:
import sqlite3, pandas as pd

conn = sqlite3.connect("scopus.db")

SQL = """
SELECT
  file_id,
  year,

  -- head: title, abstracts (raw)
  json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-title"') AS citation_title,
  json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.abstracts')        AS abstracts,

  -- source bits
  json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publisher.publishername') AS publishername,
  json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.sourcetitle')             AS sourcetitle,
  json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate."date-text"."$"') AS date_text,

  -- EXACT DOI field
  json_extract(raw_json,'$."abstracts-retrieval-response".item."item-info"."itemidlist"."ce:doi"') AS "ce:doi",

  -- reference count
  json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.tail.bibliography."@refcount"') AS refcount,

  -- NEW: cited-by count (coredata may be under item or top-level)
  CAST(
    COALESCE(
      json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."citedby-count"'),
      json_extract(raw_json,'$."abstracts-retrieval-response".coredata."citedby-count"')
    ) AS INTEGER
  ) AS citedby_count,

  -- creators as ce:indexed-name (single string)
  (
    SELECT group_concat(
             COALESCE(
               json_extract(a.value,'$."preferred-name"."ce:indexed-name"'),
               json_extract(a.value,'$."ce:indexed-name"')
             ), '; '
           )
    FROM json_each(
           COALESCE(
             json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'),
             json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author')
           )
         ) AS a
  ) AS creators_indexed_str,

  -- affiliations (organizations) from author-group, de-duplicated in a subquery
  (
    SELECT group_concat(org_name, '; ')
    FROM (
      SELECT DISTINCT
        CASE json_type(o.value)
          WHEN 'text'   THEN json_extract(o.value,'$')         -- plain string
          WHEN 'object' THEN json_extract(o.value,'$."$"')     -- {"$": "Org Name"}
          ELSE NULL
        END AS org_name
      FROM json_each(
             COALESCE(
               json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."author-group"'),
               json_array()
             )
           ) AS g
      LEFT JOIN json_each(
        CASE json_type(g.value,'$.affiliation.organization')
          WHEN 'array'  THEN json_extract(g.value,'$.affiliation.organization')
          WHEN 'object' THEN json_array(json_extract(g.value,'$.affiliation.organization'))
          WHEN 'text'   THEN json_array(json_extract(g.value,'$.affiliation.organization'))
          ELSE json_array()
        END
      ) AS o
      WHERE
        CASE json_type(o.value)
          WHEN 'text'   THEN json_extract(o.value,'$')
          WHEN 'object' THEN json_extract(o.value,'$."$"')
          ELSE NULL
        END IS NOT NULL
    )
  ) AS affiliations

FROM papers_raw
ORDER BY year, file_id;
"""

df = pd.read_sql_query(SQL, conn)
df.head()


,file_id,year,citation_title,abstracts,publishername,sourcetitle,date_text,ce:doi,refcount,citedby_count,creators_indexed_str,affiliations
0,201800000,2018,Public health and international epidemiology f...,None,Springer International Publishing,"Radiology in Global Health: Strategies, Implem...",31 December 2018,None,76,1.0,Pongpirul K.,Department of Preventive and Social Medicine; ...
1,201800001,2018,Flexible Printed Active Antenna for Digital Te...,"© 2018 The Institute of Electronics, Informati...",Institute of Electrical and Electronics Engine...,Progress in Electromagnetics Research Symposium,31 December 2018,None,4,1.0,Pratumsiri T.,None
2,201800002,2018,Parametric study of hydrogen production via so...,© 2018 Elsevier LtdComputational fluid dynamic...,Elsevier Ltd,Chemical Engineering Science,None,None,42,21.0,Phuakpunk K.,Center of Excellence in Catalysis and Catalyti...
3,201800003,2018,Superhydrophobic coating from fluoroalkylsilan...,© 2018 Elsevier B.V. A superhydrophobic/supero...,Elsevier B.V.,Applied Surface Science,None,None,45,37.0,Saengkaew J.,Department of Chemistry; Faculty of Science an...
4,201800004,2018,Electrochemical impedance-based DNA sensor usi...,© 2018 Elsevier B.V. A label-free electrochemi...,Elsevier B.V.,Analytica Chimica Acta,None,None,55,68.0,Teengam P.,Program in Petrochemistry; Faculty of Science;...
